In [10]:
import pandas as pd
import numpy as np
import MICE
import iMICE

#convert nominal
def processAge(x):
    if x >= 0 and x < 0.5:
        return 0
    elif x >= 0.5 and x <= 0.69:
        return 1
    elif x >= 0.7 and x <= 0.89:
        return 2
    elif x >= 0.9:
        return 3

df = pd.read_csv('Admission_Predict_Ver1.1.csv')
df['Chance']=df.iloc[:,8].values.copy()
df_1=df.copy()
df_1 = df_1.drop(df_1.columns[[9]],axis = 1)
df_1['ChanceCode'] = df['Chance'].map(processAge)

X = df_1.iloc[:, [1,2,6]].values  
y = df_1.iloc[:, 9].values

from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 0)

#actual
actual=np.concatenate((y_train,y_test),axis=0)
r1=len(y_train)

#y_test
df_2n = pd.DataFrame({'Chance':y_test[:]})
df_2n.at[:,'Chance']=np.nan
y_test_n=df_2n.iloc[:,0].values

y_train_1=np.concatenate((y_train,y_test_n),axis=0)
X_train_1=np.concatenate((X_train,X_test),axis=0)

df_2 = pd.DataFrame({'ChanceOfAdmit':y_train_1[:],'GREScore':X_train_1[:,0],'TOEFLScore':X_train_1[:,1],'CGPA':X_train_1[:,2]})


# MICE

In [12]:
mice_result = MICE.MICE(verbose=False, init_fill_method="median", impute_type="pmm", n_imputations=7).complete(np.matrix(df_2))

df_3 = pd.DataFrame(mice_result, columns = ['ChanceOfAdmit', 'GREScore', 'TOEFLScore', 'CGPA'])
predict = df_3['ChanceOfAdmit']

predict = np.array(predict)

predict = np.array(predict)

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix  
print(accuracy_score(y_test, predict[r1:]))
print(confusion_matrix(actual[r1:], predict[r1:]))

0.65
[[ 3  3  0  0]
 [ 9 24  8  0]
 [ 2  8 30  5]
 [ 0  0  0  8]]


# iMICE

In [13]:
imice_result = iMICE.iMICE(verbose=False, init_fill_method="median", impute_type="pmm", n_imputations=7).complete(np.matrix(df_2))

df_3i = pd.DataFrame(imice_result, columns = ['ChanceOfAdmit', 'GREScore', 'TOEFLScore', 'CGPA'])
predicti = df_3i['ChanceOfAdmit'].map(lambda x: int(x))

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix  
print(accuracy_score(y_test, predicti[r1:]))
print(confusion_matrix(actual[r1:], predicti[r1:]))

0.72
[[ 3  2  1  0]
 [ 2 27 12  0]
 [ 0  9 35  1]
 [ 0  0  1  7]]


# KNN

In [14]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(3, weights='distance')
trained_model = clf.fit(X_train, y_train)
y_pred = trained_model.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.78
[[ 3  1  2  0]
 [ 2 32  7  0]
 [ 0  7 36  2]
 [ 0  0  1  7]]


# Random Forest

In [15]:
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier

regressor = RandomForestClassifier(n_estimators=20, random_state=0)  
regressor.fit(X_train, y_train)  
y_pred = regressor.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.73
[[ 2  4  0  0]
 [ 2 30  9  0]
 [ 0  8 34  3]
 [ 0  0  1  7]]
